In [1]:
# import sys
# sys.path.append('/kaggle/input/qwertttt')

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import torch

import transformers
from transformers import BertTokenizer
from typing import *

from MEOW_Models.MT_models import MEOW_MTM
from MEOW_Models.Kernel_model import BertWithoutEmbedding

from MEOW_Utils.Data_utils import*
from MEOW_Utils.Training_utils import*

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 16
EPOCH_NUM = 10

INPUT_FILE_PATH_SQuAD = r'C:\Users\Administrator\codeblocks_workspace\MEOW\Dataset_infile\SQuAD.csv'
# INPUT_FILE_PATH_SQuAD = r'/kaggle/input/qwertttt/Data_file/SQuAD.csv'

SQuAD_DATASIZE = 100

TEST_SIZE = 0.3
PRETRAINED_MODULE_NAME = 'bert-base-uncased'

In [2]:
#處理好 dataframe
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODULE_NAME)
df_SQuAD = get_dataframe(INPUT_FILE_PATH_SQuAD, 'SQuAD', tokenizer, SQuAD_DATASIZE)

df_SQuAD.head()

c:\Users\Administrator\codeblocks_workspace\MEOW\MEOW_Utils\Data_utils.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['TKstart'][i], df_train['TKend'][i] = count_the_TKbeg_and_TKend(df_train.iloc[i]['context'], df_train.iloc[i]['answer_start'], df_train.iloc[i]['text'], tokenizer)


,index,question,context,text,answer_start,label,TKstart,TKend,SEP_ind
0,5a35d653788daf001a5f86a1,What are short round bedrock knobs?,Some rock formations in the path of a glacier ...,,-1,0,0,0,131
1,5acd449507355d001abf3b57,MAPS is the former name of which satellite?,BeiDou-2 (formerly known as COMPASS) is not an...,,-1,0,0,0,94
2,5acf90a577cf76001a685295,Who helped the rest of the world use the word ...,Historical narratives of manga tend to focus e...,,-1,0,0,0,303
3,571aa29f10f8ca140030520e,"Along with the Punjab, what did Muhammad bin W...","In the year 712, Muhammad bin Qasim, an Umayya...",Sindh,132,1,32,32,158
4,5a2ef0e3a83784001a7d25ad,What has maintained a strong presence in all t...,Contemporary a cappella includes many vocal gr...,,-1,0,0,0,145


In [2]:
l = "sfdasf"
l.lower

<function str.lower()>

In [3]:
bert_kernel_model = BertWithoutEmbedding.from_pretrained(PRETRAINED_MODULE_NAME)
SQuAD_databox = DataBox(
               bert_kernel_model = bert_kernel_model,
               df_Data = df_SQuAD,
               test_size = TEST_SIZE,
               tokenizer = tokenizer,
               label_nums = 2,
               batch_size = BATCH_SIZE,
               dataset_name = 'SQuAD'
               )

for i in SQuAD_databox.test_dataloader:
    0

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertWithoutEmbedding: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertWithoutEmbedding from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertWithoutEmbedding from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
MEOW_model = MEOW_MTM(bert_kernel_model, 
                      SQuAD_embedding_layer=SQuAD_databox.embedding_layer, 
                      device = DEVICE)

# H = {
#     "train_loss": [],
#     "train_acc": [],
#     "val_loss":[],
#     "val_acc": []
#     }

Training_a_epoc_data_num = len(SQuAD_databox.training_dataset)
Training_round = int(Training_a_epoc_data_num / BATCH_SIZE)

Test_a_epoc_data_num = len(SQuAD_databox.test_dataset)
Test_round = int(Test_a_epoc_data_num / BATCH_SIZE)

print(f'Per epoc round\'s num is {Training_round}')

Per epoc round's num is 4


In [ ]:
# 訓練
for epoch in range(EPOCH_NUM):
    print("the {:d} iter :".format(epoch+1))

    #### training now
    MEOW_model.train()

    SQuAD_training_loss = 0.0
    SQuAD_iter = iter(SQuAD_databox.training_dataloader)

    for i in range(Training_round):
        # SQuAD
        loss = Training(MEOW_model, SQuAD_iter, DEVICE, dataset_name='SQuAD')
        # print("the SQuAD loss is : ", loss.item())
        SQuAD_training_loss += loss.item()

    
    SQuAD_avg_loss = SQuAD_training_loss / Training_round
    SQuAD_databox.H['train_loss'].append(SQuAD_avg_loss)
    SQuAD_f1_score = count_F1_score(MEOW_model, SQuAD_databox.df_train, tokenizer, DEVICE)

    print("SQuAD train loss: {:.6f}".format(SQuAD_avg_loss))
    print('')


    #### test now
    MEOW_model.eval()
    SQuAD_test_loss = 0.0
    SQuAD_iter = iter(SQuAD_databox.test_dataloader)

    for i in range(Test_round):
        # SQuAD
        loss = Test(MEOW_model, SQuAD_iter, DEVICE, dataset_name='SQuAD')
        # print("the SQuAD loss is : ", loss.item())
        SQuAD_test_loss += loss.item()


    SQuAD_avg_loss = SQuAD_test_loss / Test_round
    SQuAD_databox.H['test_loss'].append(SQuAD_avg_loss)
    SQuAD_f1_score = count_F1_score(MEOW_model, SQuAD_databox.df_test, tokenizer, DEVICE)
    
    print("SQuAD test loss: {:.6f}, SQuAD f1 score: {:.4f}".format(SQuAD_avg_loss, SQuAD_f1_score))
    print('')

the 1 iter :


RuntimeError: CUDA out of memory. Tried to allocate 150.00 MiB (GPU 0; 4.00 GiB total capacity; 3.37 GiB already allocated; 0 bytes free; 3.43 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [5]:
SQuAD_f1_score = count_F1_score(MEOW_model, SQuAD_databox.df_train, tokenizer, DEVICE)